In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [16]:
using BedsData
using ForecastData
using GeographicData

In [4]:
using ReusableResourceAllocation

In [5]:
ne_states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
pct_beds_available = 0.25
travel_threshold_hours = 4.0;

In [6]:
beds = compute_beds(ne_states, pct_beds_available=pct_beds_available);
forecast_start, forecast_net = compute_ihme_forecast_net(start_date, end_date, ne_states);
adj = compute_adjacencies(ne_states, hrs=travel_threshold_hours);

In [7]:
model = patient_allocation(
    beds,
    forecast_start,
    forecast_net,
    adj,
    send_new_only=true,
    send_wait_period=10,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1395 rows, 4752 columns and 146931 nonzeros
Model fingerprint: 0xe64235ac
Model has 330 SOS constraints
Variable types: 4752 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 1e+04]
Found heuristic solution: objective 6.820000e+11
Presolve removed 546 rows and 3003 columns
Presolve time: 0.28s
Presolved: 849 rows, 1749 columns, 49364 nonzeros
Presolved model has 92 SOS constraint(s)
Found heuristic solution: objective 107644.47627
Variable types: 1649 continuous, 100 integer (100 binary)

Root relaxation: objective 1.024136e-01, 215 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [8]:
overflow = i -> sum(max.(0,
    forecast_start[i]
    + sum(forecast_net[i,1:t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,1:t])
    - beds[i])
    for t=1:size(sent,3)
);

In [9]:
summary = DataFrame(
    state=ne_states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:length(ne_states)),
)

,state,total_sent,total_received,overflow
,String,Float64,Float64,Float64
1,CT,3570.35,0.0,0.000732422
2,DE,0.0,497.868,1.90735e-6
3,MA,1398.9,0.0,0.00012207
4,MD,0.0,1344.74,0.0
5,ME,0.0,825.585,1.90735e-6
6,NH,0.0,693.241,0.0
7,NJ,1619.5,1424.55,0.000244141
8,NY,3452.37,1961.57,0.000976562
9,PA,0.0,3249.73,0.0


In [10]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(ne_states))
insertcols!(sent_matrix, 1, :state => ne_states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,210.102,0.0,0.0,558.656,482.455,940.598,1327.82,0.0,0.0,50.7181
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,168.113,178.631,463.463,588.691,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,213.601,0.0,510.956,0.0,0.0,0.0,0.0,894.94,0.0,0.0
8,NY,0.0,72.66,0.0,833.788,0.0,0.0,0.0,0.0,2354.79,0.0,191.134
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(max.(0, forecast_net), dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(ne_states));
# CSV.write("patient_sent_matrix.csv", sent_vis_matrix);

In [12]:
n_days = size(sent,3)
outcomes = DataFrame()
for (i,s) in enumerate(ne_states)
    single_state_outcome = DataFrame(
        state=fill(s, n_days),
        day=start_date .+ Dates.Day.(0:n_days-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        net_patients=forecast_net[i,:],
        total_patients=[forecast_start[i] + sum(forecast_net[i,1:t]) - sum(sent[i,:,1:t]) + sum(sent[:,i,1:t]) for t in 1:n_days],
        capacity=fill(beds[i], n_days),
        overflow=[max(0, forecast_start[i] + sum(forecast_net[i,1:t]) - sum(sent[i,:,1:t-1]) + sum(sent[:,i,1:t]) - beds[i]) for t in 1:n_days],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(ne_states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:n_days],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(ne_states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:n_days],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,net_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,CT,2020-04-01,41.2727,0.0,41.2727,744.451,2159.75,0.0,"[(""VT"", 41.2727)]",[]
2,DE,2020-04-01,0.0,213.601,1.51812,261.05,572.5,0.0,[],"[(""NJ"", 213.601)]"
3,MA,2020-04-01,39.803,0.0,39.803,557.744,3790.75,0.0,"[(""NH"", 39.803)]",[]
4,MD,2020-04-01,0.0,0.0,147.355,575.461,2793.5,0.0,[],[]
5,ME,2020-04-01,0.0,2.01424,-2.0,37.5482,887.5,0.0,[],"[(""RI"", 2.01424)]"
6,NH,2020-04-01,0.0,39.803,-3.65505,69.8986,755.0,0.0,[],"[(""MA"", 39.803)]"
7,NJ,2020-04-01,213.601,0.0,213.601,3667.08,6239.5,0.0,"[(""DE"", 213.601)]",[]
8,NY,2020-04-01,832.831,0.0,832.831,9692.93,13039.5,0.0,"[(""PA"", 832.831)]",[]
9,PA,2020-04-01,0.0,832.831,130.308,1724.93,9372.25,0.0,[],"[(""NY"", 832.831)]"


In [13]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,net_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,NJ,2020-04-01,213.601,0.0,213.601,3667.08,6239.5,0.0,"[(""DE"", 213.601)]",[]
2,NJ,2020-04-02,0.0,0.0,774.71,4441.79,6239.5,0.0,[],[]
3,NJ,2020-04-03,0.0,0.0,819.75,5261.54,6239.5,0.0,[],[]
4,NJ,2020-04-04,228.243,0.0,228.243,5261.54,6239.5,0.0,"[(""PA"", 228.243)]",[]
5,NJ,2020-04-05,241.428,0.0,484.768,5504.88,6239.5,0.0,"[(""PA"", 241.428)]",[]
6,NJ,2020-04-06,425.269,0.0,425.269,5504.88,6239.5,0.0,"[(""PA"", 425.269)]",[]
7,NJ,2020-04-07,466.18,0.0,466.18,5504.88,6239.5,0.0,"[(""MD"", 466.18)]",[]
8,NJ,2020-04-08,44.7759,0.0,734.62,6194.72,6239.5,0.0,"[(""MD"", 44.7759)]",[]
9,NJ,2020-04-09,0.0,0.0,44.7757,6239.5,6239.5,0.0,[],[]


In [14]:
println("Connectivity:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(adj)))

Connectivity:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["CT", "DE", "MA", "ME", "NH", "NJ", "NY", "VT"]
  "NJ" => ["CT", "DE", "MA", "MD", "NY", "PA", "RI"]
  "DE" => ["CT", "MD", "NJ", "NY", "PA", "RI"]
  "MD" => ["DE", "NJ", "NY", "PA"]
  "NH" => ["CT", "MA", "ME", "NY", "RI"]
  "NY" => ["CT", "DE", "MA", "MD", "NH", "NJ", "PA", "RI", "VT"]
  "ME" => ["CT", "MA", "NH", "RI", "VT"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "MA" => ["CT", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "PA" => ["DE", "MD", "NJ", "NY"]
  "VT" => ["CT", "MA", "ME", "NY", "RI"]